### Pricing Feasibility & Inventory-Aware Optimization
The objective of this notebook is to evaluate whether revenue-optimizing pricing strategies are operationally feasible given inventory constraints.

This notebook:

- Combines pricing scenarios + demand predictions

- Introduces inventory limits

- Quantifies lost revenue due to stockouts

- Identifies safe vs risky pricing decisions

This ensures pricing decisions are profitable AND executable.

The Real Business Question is

Even if a price generates high demand and revenue, can we actually fulfill that demand with available inventory?

#### Importing Libraries

In [ ]:
import pandas as pd #importing pandas for data manipulation
import numpy as np #importing numpy for mathematical calculations

Step 1 — Load Data & Pricing Scenarios

We reuse the Day 5 dataset that already contains:

- Pricing scenarios

- Predicted demand

- Revenue calculations

In [ ]:
df=pd.read_csv("ecommerce_pricing_featured.csv")
df.head()

,product_id,category,base_price,competitor_price,promotion_flag,season,customer_segment,day_of_week,channel,current_price,...,price_plus_5,price_plus_10,demand_baseline,demand_minus_10,demand_plus_5,demand_plus_10,revenue_baseline,revenue_minus_10,revenue_plus_5,revenue_plus_10
0,1102,Electronics,153.93,23.94,True,Fall,Regular,Tue,Web,111.99,...,117.5895,123.189,719.265257,793.506663,682.144553,645.023850,80550.516078,79978.330043,80213.036964,79459.843097
1,1435,Home,190.17,250.84,False,Spring,Premium,Sat,Mobile App,190.17,...,199.6785,209.187,200.987452,327.056638,137.952858,74.918265,38221.783673,55976.724816,27546.219808,15671.927082
2,1860,Electronics,166.57,257.71,False,Summer,Regular,Mon,Web,166.57,...,174.8985,183.227,357.438669,467.862734,302.226637,247.014604,59538.559139,70138.706071,52858.985435,45259.744907
3,1270,Electronics,215.97,238.38,False,Winter,Regular,Mon,Web,215.97,...,226.7685,237.567,29.951798,173.124550,-41.634578,-113.220954,6468.689904,33650.738241,-9441.410709,-26897.562284
4,1106,Fashion,102.55,255.81,True,Spring,Regular,Mon,Web,81.71,...,85.7955,89.881,920.000124,974.168043,892.916164,865.832204,75173.210125,71639.343742,76608.188764,77821.864368


####Define Inventory Assumptions

Since we don’t have live warehouse data, we simulate realistic inventory availability.

Business Assumptions:

- Inventory is limited

- Replenishment is not instant

- Excess demand results in lost sales

- Inventory levels vary across products

In [ ]:
np.random.seed(42)
df['inventory_available']=np.random.randint(low=500, high=1200, size=len(df))

This simulates real stock variability.

Step 3 — Fulfilled Demand Logic

- In real retail operations, demand does not equal sales. Sales are constrained by available inventory. This step enforces the fundamental retail rule that sales cannot exceed stock on hand. Any demand beyond inventory capacity results in lost sales.

In [ ]:
def fulfilled_demand(predicted, inventory):
  return np.minimum(predicted, inventory)

#### Step 4 — Compute Feasible Demand per Scenario

In [ ]:
df['fulfilled_baseline']= fulfilled_demand(df['demand_baseline'], df['inventory_available'])
df['fulfilled_minus_10']= fulfilled_demand(df['demand_minus_10'], df['inventory_available'])
df['fulfilled_plus_5']= fulfilled_demand(df['demand_plus_5'], df['inventory_available'])
df['fulfilled_plus_10']= fulfilled_demand(df['demand_plus_10'], df['inventory_available'])

This step converts predicted demand into feasible sales by applying inventory constraints. For each pricing scenario, demand is capped at available inventory levels, ensuring that subsequent revenue calculations reflect what can actually be fulfilled rather than theoretical demand.

### Step 5 — Calculate Feasible Revenue

In [ ]:
df['feasible_revenue_baseline']= (df['current_price']* df['fulfilled_baseline'])
df['feasible_revenue_minus_10']= (df['price_minus_10']*df['fulfilled_minus_10'])
df['feasible_revenue_plus_5']= (df['price_plus_5']* df['fulfilled_plus_5'])
df['feasible_revenue_plus_10']= (df['price_plus_10']* df['fulfilled_plus_10'])

Feasible revenue is calculated using fulfilled demand rather than predicted demand. This ensures that revenue reflects operational constraints and excludes sales that cannot be fulfilled due to inventory shortages.

### Step 6 — Stockout Risk Identification

Stockout risk is defined as the proportion of products where predicted demand exceeds available inventory under a given pricing scenario. This allows pricing strategies to be evaluated not only on revenue potential, but also on operational feasibility.

In [ ]:
df['stock_minus_10']= (df['demand_minus_10']>df['inventory_available'])
df['stock_plus_5']= (df['demand_plus_5']>df['inventory_available'])
df['stock_plus_10']= (df['demand_plus_10']>df['inventory_available'])

### Calculate Stockout Rate

In [ ]:
stockout_rate_minus_10 = df['stock_minus_10'].mean()
stockout_rate_plus_5 = df['stock_plus_5'].mean()
stockout_rate_plus_10 = df['stock_plus_10'].mean()

In [ ]:
def risk_label(rate):
  if rate < 0.10:
    return "Low"
  elif rate < 0.30:
    return "Medium"
  else:
    return "High"


### Step 7 — Executive Scenario Comparison
This table contrasts theoretical revenue (model-predicted demand × price) with feasible revenue (inventory-constrained sales × price).
It highlights that revenue-maximizing pricing strategies may be operationally infeasible due to inventory limitations.

In [ ]:
scenario_summary= pd.DataFrame({'Scenario': ['Baseline', '-10% Price', '5% Price', '10% Price'],
                                'Theoretical_Revenue': [df['revenue_baseline'].sum(), df['revenue_minus_10'].sum(), df['revenue_plus_5'].sum(), df['revenue_plus_10'].sum()],
                                'Feasible_Revenue': [df['feasible_revenue_baseline'].sum(), df['feasible_revenue_minus_10'].sum(), df['feasible_revenue_plus_5'].sum(), df['feasible_revenue_plus_10'].sum()],
                                'Stockout_Risk': ['Low', risk_label(stockout_rate_minus_10), risk_label(stockout_rate_plus_5), risk_label(stockout_rate_plus_10)]})
scenario_summary

,Scenario,Theoretical_Revenue,Feasible_Revenue,Stockout_Risk
0,Baseline,3.209570e+08,2.733505e+08,Low
1,-10% Price,4.890323e+08,4.355521e+08,High
2,5% Price,2.202385e+08,1.751865e+08,Medium
3,10% Price,1.083993e+08,6.568919e+07,Medium


### Interpretation:

-10% Price

- Looks amazing theoretically

- Loses value due to stockouts

- High operational risk → not safe

Baseline

- Stable

- Lower revenue but low risk

+5% Price

- Moderate revenue

- Medium risk

- Often the best executive compromise

+10% Price

- Demand suppressed

- No stockouts

- But revenue suffers

### Final Business Recommendations

Pricing Strategy

- Avoid aggressive discounts unless inventory buffers are increased in advance

- Prefer moderate price adjustments that balance revenue growth with fulfillment stability

Inventory Coordination

- Align promotional campaigns with replenishment cycles

- Use demand forecasts to proactively adjust inventory before price changes

Executive Takeaway

The most profitable price is not the one that maximizes demand, but the one that maximizes fulfilled revenue.